# UCI Machine Learning Repository: Online Retail Dataset

## Dataset Information

Online shopping platforms often recommend similar products to their customers for more sales. In order to do such recommendations, they have to first understand the similarity among their products from past sales record. This dataset contains all the transactions occurring between 01/12/2010 and 09/12/2011.

## Text Analysis: Topic Modelling

### Objective 1:
The number of distinct products in this dataset can be as large as the number of distinct words in a corpus, while one customer only purchases a small portion of products, just like an article only uses a small portion of words from the corpus. Therefore, we may use topic models to model groups of products. In this case, we will construct a matrix with rows representing each customer, and columns representing each product. The value at position (i, j) in the matrix is the quantity of product j purchased by customer i. Data preprocessing to the original data will have to be first performed to filter invalid transactions. Next, we will apply Latent Dirichlet Allocation (LDA) on this matrix to find the grouping of products.

### Objective 2:
Based on the descriptions of the products provided by the dataset, we will select some representative words from the descriptions for each group of products. To compute the conditional probability of a word w given a group g, we can marginalize over all products x, i.e., 𝑝(𝑤|𝑔)=Σ𝑝(𝑤|𝑥)∙𝑝(𝑥|𝑔)

## Objective 1

### 1.0 Loading Packages and Reading Dataset

Prior to building our models, we will first perform preliminary data exploration to understand our data and to subsequently conduct the necessary data pre-processing.

We will first load the necessary packages, set our view of lists as tabular data (to allow us to view our data easily) and to read our dataset.

In [3]:
# Loading the required packages
import numpy as np
import pandas as pd
import os

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 15)


# Reading our Dataset
retail = pd.read_csv('onlineRetail.csv', encoding = 'iso-8859-1', dtype = {'CustomerID': str})

### 1.1 Data Exploration & Data Pre-Processing

In order to understand the necessary data pre-processing that has to be done, we will first take a glimpse at our dataset.

#### 1.1.1 Data Shape

In [4]:
print("The dataset has", retail.shape[0], "rows and", retail.shape[1], "columns.")

The dataset has 541909 rows and 8 columns.


#### 1.1.2 Data Head

In [5]:
retail.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


#### 1.1.3 Data Tail

In [6]:
retail.tail(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,12/9/2011 12:50,4.95,12680,France


#### 1.1.4 Data Count and Type

In [7]:
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null object
UnitPrice      541909 non-null float64
CustomerID     406829 non-null object
Country        541909 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 33.1+ MB


Based on Section 1.1.1.1. Data Shape, we know that the dataset has 541,909 rows. In this case, we can see from the retail.info above that both columns "Description" and "Customer ID" has missing values. The table below thus shows the total number of missing values from both "Description" and "Customer ID" columns:

In [8]:
num_row, num_col = retail.shape

retail_null = []

for i in range(num_col):
    null_values = [[retail.iloc[:, i].isnull().sum()]]
    for results in null_values:
        retail_null.append(results)

pd.DataFrame(np.transpose(retail_null), columns = retail.columns, index = ["miss value"])

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
miss value,0,0,1454,0,0,0,135080,0


Since we are interested in first building a matrix representing products bought by each customer, it would not be useful to include information for transactional information with customer ID. In this case, we will remove these rows.

#### 1.1.5. Removing Rows with Missing Value in "CustomerID" Column

In [9]:
retail = retail.dropna(subset=['CustomerID'])

#### 1.1.6. Checking for Missing Values in "Description" Column

Once we have removed all rows with missing values in "CustomerID" column, we want to recheck if there are still any more missing values left in the dataset.

In [10]:
num_row, num_col = retail.shape

retail_null = []

for i in range(num_col):
    null_values = [[retail.iloc[:, i].isnull().sum()]]
    for results in null_values:
        retail_null.append(results)

pd.DataFrame(np.transpose(retail_null), columns = retail.columns, index = ["miss value"])

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
miss value,0,0,0,0,0,0,0,0


As we can see from the above, all missing values in the "Description" column detected earlier has also been removed when the rows containing missing values in "CustomerID" were removed.

We want to now take another look at our resulting dataset shape as below:

In [11]:
print("After removing missing values, the dataset now has", retail.shape[0], "rows and", retail.shape[1], "columns.")

After removing missing values, the dataset now has 406829 rows and 8 columns.


#### 1.1.7. Data Exploration on Numeric Columns "Quantity" and "UnitPrice"

Based on the table above, we can see that our dataset has 2 columns with numeric values:
1. Quantity
2. UnitPrice

We will conduct further exploration on these two variables to understand their distribution

In [12]:
retail.describe()

,Quantity,UnitPrice
count,406829.000000,406829.000000
mean,12.061303,3.460471
std,248.693370,69.315162
min,-80995.000000,0.000000
25%,2.000000,1.250000
50%,5.000000,1.950000
75%,12.000000,3.750000
max,80995.000000,38970.000000


It is curious to note that there are negative values for the min of Quantity. Seeing that this dataset contains online retail transactions, the negative values might refer to cases of cancellation or returned orders. 

Additionally, there are observations where unit price is 0. 

We will now conduct further investigation by first taking a look at a sample of datasets with 'Quantity' with negative values, before investigating observations with unit price equals to 0.

#### 1.1.8. Investigating Negative Values in 'Quantity' Columns

In [13]:
retail[retail['Quantity'] <0][0:5]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,12/1/2010 9:41,27.50,14527,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,12/1/2010 9:49,4.65,15311,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,12/1/2010 10:24,1.65,17548,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548,United Kingdom


Based on the two tables above, there are two things to take note:

##### 1. Cancelled Orders
It seems like transactions with negative Quantity refers to transactions that are cancelled. This seems to commensurate with the Invoice Number, where it is prefixed with "C", which could stand for "Cancelled". Since our objective is to understand the similarity of products bought by customers from past sales record, products that are not ultimately sold would not be useful to be included as part of our analysis.

##### 2. Discounted Transactions and Non-Customer Transactional Stock Code Name
In the first row of the first table, we can see that negative quantity seems to commensurate with Discounts as well, and the product name, or Stock Code of the product is reflected as "D", which most probably stands for "Discount". In this case, it is not useful to include this information as part of our analysis.

Since discounted transaction is represented by "D" in the Stock Code, there might be other rows of observations that do not necessary relate to customer purchase transactions. Additionally, from section 1.1.1.2. above, where we have viewed a sample of the data head, we can see that Stock Codes are typically a 5-digits or 6-digits code with a suffix at the end in some instances. 

Therefore, with the assumption that all non-customer related transactions will not have stock code that commensurate with this 5-character or 6-character alphanumeric nominal value, we will perform further investigation on observations with Stock Code that do not follow such patterns.

#### 1.1.9 Removing Cancelled Orders From Dataset

In [14]:
retail = retail[retail['Quantity']>0]

#### 1.1.10 Removing Observations with "D" in "StockCode" Column

In [15]:
retail = retail[retail['StockCode']!='D']

#### 1.1.11 Identifying Possible Non-Customer Related Transaction Stock Codes

Once we have removed the cancelled and discounted transactions from the dataset, we will now proceed to further investigate on observations that might not be related to customer transactions. We will conduct this by identifying Stock Codes that do not follow the conventional unique identifier of 5 to 6 alphanumeric characters.

Thereafter, we will filter out these observations and determine if they should be included or excluded from our final analysis.

In [16]:
non_trans_stock = []
for i in [1,2,3,4,7,8,9,10,11,12]:
    non_trans_values = retail[retail['StockCode'].astype(str).str.len() == i]
    non_trans_set = [set(non_trans_values['StockCode'])]
    if non_trans_set != [set()]:
        print("For column 'StockCode' with length", str(i), ", the Stock Code(s) that might not be related to customer transactions is / are", non_trans_set, "\n")

For column 'StockCode' with length 1 , the Stock Code(s) that might not be related to customer transactions is / are [{'M'}] 

For column 'StockCode' with length 2 , the Stock Code(s) that might not be related to customer transactions is / are [{'C2'}] 

For column 'StockCode' with length 3 , the Stock Code(s) that might not be related to customer transactions is / are [{'DOT'}] 

For column 'StockCode' with length 4 , the Stock Code(s) that might not be related to customer transactions is / are [{'PADS', 'POST'}] 

For column 'StockCode' with length 7 , the Stock Code(s) that might not be related to customer transactions is / are [{'15056BL'}] 

For column 'StockCode' with length 12 , the Stock Code(s) that might not be related to customer transactions is / are [{'BANK CHARGES'}] 



##### Observing 'StockCode' with value 'M'

In [17]:
retail[retail['StockCode'] == "M"].head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
2239,536569,M,Manual,1,12/1/2010 15:35,1.25,16274,United Kingdom
2250,536569,M,Manual,1,12/1/2010 15:35,18.95,16274,United Kingdom
6798,536981,M,Manual,2,12/3/2010 14:26,0.85,14723,United Kingdom
7976,537077,M,Manual,12,12/5/2010 11:59,0.42,17062,United Kingdom
8530,537137,M,Manual,36,12/5/2010 12:43,0.85,16327,United Kingdom


From above, it seems like column 'StockCode' with value 'M' relates to transactions that are manually keyed into the system, without assigning product code or description. Since there are no product code or description of the product for us to analyse with, we will be dropping these observations from our dataset.

##### Dropping Observations with value 'M' in column 'StockCode'

In [18]:
retail = retail[retail['StockCode']!='M']

##### Observing 'StockCode' with value 'C2'

In [19]:
retail[retail['StockCode'] == "C2"].head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1423,536540,C2,CARRIAGE,1,12/1/2010 14:05,50.0,14911,EIRE
12119,537368,C2,CARRIAGE,1,12/6/2010 12:40,50.0,14911,EIRE
12452,537378,C2,CARRIAGE,1,12/6/2010 13:06,50.0,14911,EIRE
19975,537963,C2,CARRIAGE,1,12/9/2010 11:30,50.0,13369,United Kingdom
20016,538002,C2,CARRIAGE,1,12/9/2010 11:48,50.0,14932,Channel Islands


From above, it seems like column 'StockCode' with value 'C2' relates to transportation charges. Since this is not related to transactions of product(s), there are no product code or description of the product for us to analyse with, we will be dropping these observations from our dataset.

##### Dropping Observations with value 'C2' in column 'StockCode'

In [20]:
retail = retail[retail['StockCode']!='C2']

##### Observing 'StockCode' with value 'DOT'

In [21]:
retail[retail['StockCode'] == "DOT"].head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
317507,564764,DOT,DOTCOM POSTAGE,1,8/30/2011 10:49,11.17,14096,United Kingdom
324002,565383,DOT,DOTCOM POSTAGE,1,9/2/2011 15:45,16.46,14096,United Kingdom
333755,566217,DOT,DOTCOM POSTAGE,1,9/9/2011 15:17,13.16,14096,United Kingdom
338829,566566,DOT,DOTCOM POSTAGE,1,9/13/2011 12:32,85.58,14096,United Kingdom
350599,567656,DOT,DOTCOM POSTAGE,1,9/21/2011 14:40,878.55,14096,United Kingdom


Similar to observations with Stock Code = 'C2', it seems like column 'StockCode' with value 'DOT' relates to transportation charges. Since this is not related to transactions of product(s), there are no product code or description of the product for us to analyse with, we will be dropping these observations from our dataset.

##### Dropping Observations with value 'DOT' in column 'StockCode'

In [22]:
retail = retail[retail['StockCode']!='DOT']

##### Observing 'StockCode' with value 'PADS'

In [23]:
retail[retail['StockCode'] == "PADS"].head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
157195,550193,PADS,PADS TO MATCH ALL CUSHIONS,1,4/15/2011 9:27,0.001,13952,United Kingdom
279045,561226,PADS,PADS TO MATCH ALL CUSHIONS,1,7/26/2011 10:13,0.001,15618,United Kingdom
358655,568158,PADS,PADS TO MATCH ALL CUSHIONS,1,9/25/2011 12:22,0.000,16133,United Kingdom
359871,568200,PADS,PADS TO MATCH ALL CUSHIONS,1,9/25/2011 14:58,0.001,16198,United Kingdom


From above, it seems like the product "PADS TO MATCH ALL CUSHIONS" is a non-product, given that the unit price of it is 0.001. Therefore, it would not be useful to include these observations in our final analysis.

##### Dropping Observations with value 'PADS' in column 'StockCode'

In [24]:
retail = retail[retail['StockCode']!='PADS']

##### Observing 'StockCode' with value 'POST'

In [25]:
retail[retail['StockCode'] == "POST"].head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
45,536370,POST,POSTAGE,3,12/1/2010 8:45,18.0,12583,France
386,536403,POST,POSTAGE,1,12/1/2010 11:27,15.0,12791,Netherlands
1123,536527,POST,POSTAGE,1,12/1/2010 13:04,18.0,12662,Germany
5073,536840,POST,POSTAGE,1,12/2/2010 18:27,18.0,12738,Germany
5258,536852,POST,POSTAGE,1,12/3/2010 9:51,18.0,12686,France


Similar to observations with Stock Code = 'C2' and 'DOT', it seems like column 'StockCode' with value 'POST' relates to transportation or postage charges. Since this is not related to transactions of product(s), there are no product code or description of the product for us to analyse with, we will be dropping these observations from our dataset.

##### Dropping Observations with value 'POST' in column 'StockCode'

In [26]:
retail = retail[retail['StockCode']!='POST']

##### Observing 'StockCode' with value '15056BL'

In [27]:
retail[retail['StockCode'] == "15056BL"].head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
132,536381,15056BL,EDWARDIAN PARASOL BLACK,2,12/1/2010 9:41,5.95,15311,United Kingdom
281,536396,15056BL,EDWARDIAN PARASOL BLACK,6,12/1/2010 10:51,4.95,17850,United Kingdom
1219,536531,15056BL,EDWARDIAN PARASOL BLACK,12,12/1/2010 13:23,5.95,15485,United Kingdom
4164,536750,15056BL,EDWARDIAN PARASOL BLACK,6,12/2/2010 14:04,4.95,17850,United Kingdom
4185,536752,15056BL,EDWARDIAN PARASOL BLACK,6,12/2/2010 14:06,4.95,17850,United Kingdom


From above, we can see that these observations do relate to transactions of products sold to customers, despite not having the conventional 5 or 6 alphanumeric character Stock Code Identifier. Therefore, we will not be dropping these observations from our dataset.

##### Observing 'StockCode' with value 'BANK CHARGES'

In [28]:
retail[retail['StockCode'] == "BANK CHARGES"].head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
4406,536779,BANK CHARGES,Bank Charges,1,12/2/2010 15:08,15.0,15823,United Kingdom
62508,541505,BANK CHARGES,Bank Charges,1,1/18/2011 15:58,15.0,15939,United Kingdom
152966,549717,BANK CHARGES,Bank Charges,1,4/11/2011 14:56,15.0,14606,United Kingdom
175275,551945,BANK CHARGES,Bank Charges,1,5/5/2011 11:09,15.0,16714,United Kingdom
327921,565735,BANK CHARGES,Bank Charges,1,9/6/2011 12:25,15.0,16904,United Kingdom


From above, it is clear that these transactions are invalid as they relate to bank charges, instead of products sold to customers. Therefore, since they will not be of value in our final analysis, we will be dropping these observations.

##### Dropping Observations with value 'BANK CHARGES' in column 'StockCode'

In [29]:
retail = retail[retail['StockCode']!='BANK CHARGES']

#### 1.1.12 Revisiting Data Exploration on Numeric Columns "Quantity" and "UnitPrice"

To recap on the data pre-processing we have done above after conducting our initial investigation on negative values in the 'Quantity' column, we have:
1. Removed observations with missing values from 'CustomerID' column
2. Removed observations with negative values from 'Quantity' column
3. Removed observations with value 'D' from 'StockCode' column
4. Removed obsercations with value 'M' from 'StockCode' column
5. Removed obsercations with value 'C2' from 'StockCode' column
6. Removed obsercations with value 'DOT' from 'StockCode' column
7. Removed obsercations with value 'PADS' from 'StockCode' column
8. Removed obsercations with value 'POST' from 'StockCode' column
9. Removed obsercations with value 'BANK CHARGES' from 'StockCode' column

We would now like to revisit the "UnitPrice" column, which we have also noticed that there were some observations with 0 unit price.

In [30]:
retail[retail['UnitPrice'] == 0].head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
9302,537197,22841,ROUND CAKE TIN VINTAGE GREEN,1,12/5/2010 14:02,0.0,12647,Germany
33576,539263,22580,ADVENT CALENDAR GINGHAM SACK,4,12/16/2010 14:36,0.0,16560,United Kingdom
40089,539722,22423,REGENCY CAKESTAND 3 TIER,10,12/21/2010 13:45,0.0,14911,EIRE
47068,540372,22090,PAPER BUNTING RETROSPOT,24,1/6/2011 16:41,0.0,13081,United Kingdom
47070,540372,22553,PLASTERS IN TIN SKULLS,24,1/6/2011 16:41,0.0,13081,United Kingdom


In [31]:
print("There are", len(retail[retail['UnitPrice'] == 0]), "observations with unit price = 0.")

There are 33 observations with unit price = 0.


Based on the above, we can see that there are 33 observations with unit price = 0. These transactions could be due to free gifts that comes with the purchase of other items. With this assumption, it would not be useful to include these observations in our analysis as these products are not technically bought by the customers, and that it is possible that they are not genuinely interested in purchasing these products in the first place.

##### Dropping Observations with value '0' in column 'UnitPrice'

In [32]:
retail = retail[retail['UnitPrice']>0]

### 1.2 Constructing Matrix Representing Products Bought by Each Customer

Since we have our cleaned dataset after performing data pre-processing above, we will now create a matrix with rows representing each customer (CustomerID), and columns representing each product (StockCode).

In [33]:
retail_cust_stock = retail[['CustomerID', 'StockCode', 'Quantity']]

retail_group_stock = retail_cust_stock.pivot_table(index='CustomerID', columns='StockCode', values='Quantity', aggfunc=np.sum)

retail_group_stock = retail_group_stock.fillna(0)

#### 1.2.1 Matrix Data Summary

Upon constructing our matrix, we will take a glimpse at the data to check how many rows and columns the newly created matrix has, and to glimpse at the head (top 5 rows) of the matrix.

In [34]:
print("The matrix has", retail_group_stock.shape[0], "rows (Customers) and", retail_group_stock.shape[1], "columns (Products).")

The matrix has 4334 rows (Customers) and 3659 columns (Products).


In [35]:
retail_group_stock.head(5)

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,...,90214S,90214T,90214U,90214V,90214W,90214Y,90214Z
CustomerID,,,,,,,,,,,,,,,
12346,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12347,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12348,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12350,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 1.3 Applying Latent Dirichlet Allocation (LDA) to Find Grouping of Products

With our newly created matrix, we will now apply LDA to the matrix to find the grouping of products.

In [36]:
from sklearn import datasets, feature_extraction, decomposition

x_stock = retail_group_stock
product_names_stock = list(x_stock)

In [37]:
n, m = retail_group_stock.shape
k = 5

lda_stock = decomposition.LatentDirichletAllocation(n_components = k, random_state = 2018)
xtr = lda_stock.fit_transform(x_stock)

topic_word_stock = lda_stock.components_
group_given_prod = topic_word_stock

#### 1.3.1 Obtaining Top 10 Most Representative Products for Each of the Five Groups

Upon applying LDA to obtain the group of products in each of the five groups, we will now take a look at the top 10 most representative product for each of the five groups. Given a product, we will calculate the probability that it will appear in each of the group. Thereafter, we will take the top 10 products with the highest probabilities in each group.

In [38]:
prob_group_given_prod = []
prodname = []

for j in range(m):
    group_given_prod[:, j] /= sum(group_given_prod[:,j])
for i in range(k):
    prob_j_stock = []
    for j in range(m):
        prob_j_stock.append((group_given_prod[i,j], j))
    
    prob_sorted_stock = sorted(prob_j_stock,reverse = True)
    for j in range(10):
        result1 = [prob_sorted_stock[j][0]]
        for results in result1:
            prob_group_given_prod.append(results)
        result2 = [product_names_stock[prob_sorted_stock[j][1]]]
        for results in result2:
            prodname.append(results)

print(len(prob_j_stock))
table_val = np.transpose(np.vstack((prodname, prob_group_given_prod)))

index_name = ["1st Most Representative Product", 
            "2nd Most Representative Product",
            "3rd Most Representative Product",
            "4th Most Representative Product",
            "5th Most Representative Product",
            "6th Most Representative Product",
            "7th Most Representative Product",
            "8th Most Representative Product",
            "9th Most Representative Product",
            "10th Most Representative Product"]

column_name = ["Stock Code", "Probability"]


group1_result = table_val[0:10]
group1_result = pd.DataFrame(group1_result, columns = column_name, index = index_name)

group2_result = table_val[10:20]
group2_result = pd.DataFrame(group2_result, columns = column_name, index = index_name)

group3_result = table_val[20:30]
group3_result = pd.DataFrame(group3_result, columns = column_name, index = index_name)

group4_result = table_val[30:40]
group4_result = pd.DataFrame(group4_result, columns = column_name, index = index_name)

group5_result = table_val[40:50]
group5_result = pd.DataFrame(group5_result, columns = column_name, index = index_name)

3659


#### 1.3.2 Obtaining Top 10 Most Representative Products for Group 1

In [39]:
group1_result

,Stock Code,Probability
1st Most Representative Product,84508A,0.9995053150938504
2nd Most Representative Product,85099C,0.9993709667437217
3rd Most Representative Product,21785,0.9980554345274136
4th Most Representative Product,84691,0.9967075872619684
5th Most Representative Product,90209C,0.9963665610949194
6th Most Representative Product,22381,0.9963167976980114
7th Most Representative Product,75131,0.9933994749617469
8th Most Representative Product,22693,0.9932020727006798
9th Most Representative Product,90209B,0.9921468740681535
10th Most Representative Product,85099F,0.9916410739155441


#### 1.3.3 Obtaining Top 10 Most Representative Products for Group 2

In [40]:
group2_result

,Stock Code,Probability
1st Most Representative Product,16219,0.9997070063884806
2nd Most Representative Product,47556B,0.9996918137335702
3rd Most Representative Product,22608,0.9996028611854265
4th Most Representative Product,21650,0.9995936433606116
5th Most Representative Product,22102,0.9994736296117158
6th Most Representative Product,16045,0.9993490133153229
7th Most Representative Product,21641,0.9992708415185326
8th Most Representative Product,16054,0.9992650057344818
9th Most Representative Product,16216,0.999264648544582
10th Most Representative Product,35648,0.9987685680648672


#### 1.3.4 Obtaining Top 10 Most Representative Products for Group 3

In [41]:
group3_result

,Stock Code,Probability
1st Most Representative Product,16014,0.9999398792095567
2nd Most Representative Product,84568,0.9999357526790894
3rd Most Representative Product,18007,0.9998629731992309
4th Most Representative Product,22041,0.9987862584991964
5th Most Representative Product,22458,0.9984348994147968
6th Most Representative Product,21898,0.9982034936917891
7th Most Representative Product,21897,0.9980148883353323
8th Most Representative Product,17003,0.9970082845880163
9th Most Representative Product,37450,0.9898435632819346
10th Most Representative Product,82551,0.989500875255165


#### 1.3.5 Obtaining Top 10 Most Representative Products for Group 4

In [42]:
group4_result

,Stock Code,Probability
1st Most Representative Product,23843,0.9999901229690166
2nd Most Representative Product,22153,0.9995214483452146
3rd Most Representative Product,22791,0.9986957180071302
4th Most Representative Product,22688,0.9986664883716871
5th Most Representative Product,84947,0.9985114627818266
6th Most Representative Product,22154,0.9979988734724914
7th Most Representative Product,22798,0.9979785025504924
8th Most Representative Product,23521,0.9979296560875668
9th Most Representative Product,22686,0.9978382434737294
10th Most Representative Product,23522,0.9976456256682543


#### 1.3.6 Obtaining Top 10 Most Representative Products for Group 5

In [43]:
group5_result

,Stock Code,Probability
1st Most Representative Product,22326,0.9998839944068846
2nd Most Representative Product,22328,0.9998562947596064
3rd Most Representative Product,23346,0.9996873502745198
4th Most Representative Product,23345,0.9996648382818186
5th Most Representative Product,22631,0.999451667023547
6th Most Representative Product,22551,0.9994014569150087
7th Most Representative Product,22613,0.999332993277912
8th Most Representative Product,23289,0.9992896069792803
9th Most Representative Product,22243,0.9992888353694789
10th Most Representative Product,23292,0.9992836533766267


## Objective Two

### 2.0 Preliminary Discussion on Product Grouping

Although we were able to obtain the grouping of products in each of the five groups, it is still difficult to interpret the unique product identifier, StockCode. Therefore, we can attempt to use the descriptions of products to determine the most representative words in each of the five groups.

In order to proceed, we first need to find the sum of probability of words appearing for each product. Thereafter, based on the probability of products appearing in each group obtained in Objective 1, we will then be able to obtain the probability of words appearing in each group and thus identify the most representative words for each group.

However, before doing so, there are several data pre-processing steps that have to be taken as indicated below:
1. Ensure that product description is consistent for each product
2. Remove unnecessary characters 
3. Remove duplicated words

### 2.1 Text Data Pre-Processing

#### 2.1.1 Checking for Inconsistent Product Description 

In [44]:
print("There are", retail['StockCode'].nunique(), "unique values of StockCode", "\n")

print("There are", retail['Description'].nunique(), "unique values of Description", "\n")

There are 3659 unique values of StockCode 

There are 3871 unique values of Description 



From the above, we can see that there are more unique values of Description than StockCode. This could mean that there are multiple descriptions for the same StockCode for some StockCodes. Therefore, we will first append these inconsistent descriptions for each of the affected StockCodes.

#### 2.1.2 Appending Inconsistent Description to Each Affected StockCode

In [45]:
from collections import defaultdict

retail_stock_desc = retail[['Description', 'StockCode']]
retail_stock_desc = np.asarray(retail_stock_desc).tolist()

res = defaultdict(list)

for v, k in retail_stock_desc:
    if v not in res[k]:
        res[k].append(v)
    else:
        res[k]

stockcode_desc_dup = [[k,v] for k, v in res.items()]
        
stockcode_desc_dup = pd.DataFrame(stockcode_desc_dup, columns = ("StockCode", "Description"))

stockcode_desc_dup.head(5)

,StockCode,Description
0,85123A,"[WHITE HANGING HEART T-LIGHT HOLDER, CREAM HAN..."
1,71053,"[WHITE METAL LANTERN, WHITE MOROCCAN METAL LAN..."
2,84406B,[CREAM CUPID HEARTS COAT HANGER]
3,84029G,[KNITTED UNION FLAG HOT WATER BOTTLE]
4,84029E,[RED WOOLLY HOTTIE WHITE HEART.]


#### 2.1.3 Removing Unnecessary Characters from Product Description

Next, after appending the inconsistent Product Description to each of the affected StockCode, we can see from the table above that the description contains many unnecessary characters such as punctuations (for example: "," and "."), and numbers (0-9) that might interfere with the finding of the most representative words for each of the five groups.

Therefore, we will attempt to remove these unnecessary characters from the "Description" column.

In [46]:
stockcode_desc_dup["Description"] = stockcode_desc_dup["Description"].astype(str)

desc_cleaned = []
for words in stockcode_desc_dup["Description"]:
    words = words.replace("[", "")
    words = words.replace("]", "")
    words = words.replace('"', "")
    words = words.replace("'", "")
    words = words.replace(",", "")
    words = words.replace(".", "")
    words = words.replace("/", "")
    words = words.replace("0", "")
    words = words.replace("1", "")
    words = words.replace("2", "")
    words = words.replace("3", "")
    words = words.replace("4", "")
    words = words.replace("5", "")
    words = words.replace("6", "")
    words = words.replace("7", "")
    words = words.replace("8", "")
    words = words.replace("9", "")
    desc_cleaned.append(words)

    
desc_cleaned = pd.DataFrame(desc_cleaned, columns = ["Description"])

desc_cleaned.head(5)

,Description
0,WHITE HANGING HEART T-LIGHT HOLDER CREAM HANGI...
1,WHITE METAL LANTERN WHITE MOROCCAN METAL LANTERN
2,CREAM CUPID HEARTS COAT HANGER
3,KNITTED UNION FLAG HOT WATER BOTTLE
4,RED WOOLLY HOTTIE WHITE HEART


As seen in the table above, the unnecessary characters have been removed from the "Description" column.

#### 2.1.4 Removing Duplicated Words

Next, once we have removed the unnecessary characters from the description, we now have duplicated words for some of the StockCode. Therefore, we want to now remove these duplicated words from the description of each of the StockCode.

In [47]:
from collections import OrderedDict
desc_cleaned['NoDupWords'] = desc_cleaned.Description.str.split().apply(lambda x: OrderedDict.fromkeys(x).keys()).str.join(' ')
desc_cleaned = desc_cleaned.drop(columns=['Description'])

desc_cleaned.head(5)

,NoDupWords
0,WHITE HANGING HEART T-LIGHT HOLDER CREAM
1,WHITE METAL LANTERN MOROCCAN
2,CREAM CUPID HEARTS COAT HANGER
3,KNITTED UNION FLAG HOT WATER BOTTLE
4,RED WOOLLY HOTTIE WHITE HEART


As seen in the table above, we have created a new column 'NoDupWords' to store all description for each StockCode without duplicated words.

### 2.2 Obtaining Probability of Words Appearing in Each Product

Once we have cleaned our text data, we can now proceed to calculate the probability of words appearing in each product.

#### 2.2.1 Extracting Words From Description

First, we have to vectorize our text to obtain the words individually.

In [48]:
count_vectorizer = feature_extraction.text.CountVectorizer(stop_words = 'english')
x1 = count_vectorizer.fit_transform(desc_cleaned['NoDupWords'])
vocab = np.asarray(count_vectorizer.get_feature_names())

pd.DataFrame(vocab, columns = ["Word"]).head(5)

,Word
0,abc
1,abstract
2,acapulco
3,account
4,acrylic


#### 2.2.2 Converting Count of Words per Product into Matrix

In [49]:
x2 = x1.toarray()

x2

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

#### 2.2.3 Calculating Total Number of Words Per Product

In [50]:
x3 = x2.sum(axis=1)
x3

array([6, 4, 5, ..., 4, 5, 4], dtype=int64)

#### 2.2.4 Calculating Probability of Words Appearing, Given Each Product

In [51]:
x4 = np.asarray(x2)/np.asarray(x3[:,None])
x4 = pd.DataFrame(x4, columns = vocab, index = stockcode_desc_dup["StockCode"])

word_given_prod = x4

word_given_prod

,abc,abstract,acapulco,account,acrylic,address,ade,...,yellowblue,yelloworange,yellowpink,youre,yuletide,zinc,zincglass
StockCode,,,,,,,,,,,,,,,
85123A,0.00,0.000000,0.00,0.00,0.000000,0.00,0.0,...,0.000000,0.00,0.0,0.00,0.00,0.000000,0.000000
71053,0.00,0.000000,0.00,0.00,0.000000,0.00,0.0,...,0.000000,0.00,0.0,0.00,0.00,0.000000,0.000000
84406B,0.00,0.000000,0.00,0.00,0.000000,0.00,0.0,...,0.000000,0.00,0.0,0.00,0.00,0.000000,0.000000
84029G,0.00,0.000000,0.00,0.00,0.000000,0.00,0.0,...,0.000000,0.00,0.0,0.00,0.00,0.000000,0.000000
84029E,0.00,0.000000,0.00,0.00,0.000000,0.00,0.0,...,0.000000,0.00,0.0,0.00,0.00,0.000000,0.000000
22752,0.00,0.000000,0.00,0.00,0.000000,0.00,0.0,...,0.000000,0.00,0.0,0.00,0.00,0.000000,0.000000
21730,0.00,0.000000,0.00,0.00,0.000000,0.00,0.0,...,0.000000,0.00,0.0,0.00,0.00,0.000000,0.000000
22633,0.00,0.000000,0.00,0.00,0.000000,0.00,0.0,...,0.000000,0.00,0.0,0.00,0.00,0.000000,0.000000
22632,0.00,0.000000,0.00,0.00,0.000000,0.00,0.0,...,0.000000,0.00,0.0,0.00,0.00,0.000000,0.000000


#### 2.2.5 Calculating the Probability of Products Appearing, Given Each Group

Based on section 1.1.3.1. above, we have obtained the probability of a product belonging in each of the five groups, across the five groups. However, in order to achieve our final objective of identifying the probability of a word appearing given a group, we need to first calculate the probability of products given each group as follow:

In [52]:
prod_given_group = topic_word_stock

prod_given_group = np.asmatrix(prod_given_group)

prod_given_group_sum = prod_given_group.sum(axis=1)

prod_given_group = pd.DataFrame(prod_given_group/prod_given_group_sum)

prod_given_group

,0,1,2,3,4,5,6,...,3652,3653,3654,3655,3656,3657,3658
0,2.087408e-04,1.808044e-06,0.000152,0.000087,0.000078,0.000574,1.475508e-05,...,0.001600,0.000040,0.000040,0.000035,0.000040,0.000008,0.000040
1,1.406345e-04,1.106879e-03,0.001053,0.000039,0.000018,0.000013,8.249494e-05,...,0.000413,0.001054,0.001054,0.001063,0.001054,0.000188,0.001054
2,6.155138e-07,1.704714e-06,0.000003,0.000685,0.000030,0.000027,4.575597e-07,...,0.000013,0.000038,0.000038,0.000033,0.000038,0.002033,0.000038
3,2.084500e-05,6.947521e-07,0.000001,0.000034,0.000013,0.000012,1.659035e-07,...,0.000005,0.000015,0.000015,0.000013,0.000015,0.000003,0.000015
4,7.862777e-04,1.256467e-05,0.000001,0.000631,0.000945,0.000756,9.360449e-04,...,0.000005,0.000016,0.000016,0.000014,0.000016,0.000003,0.000016


#### 2.2.6 Changing Probability of Product Given Group and Probability of Word Given Product into Matrix

In [53]:
prob_prod_group = np.asmatrix(prod_given_group)
prob_word_prod = np.asmatrix(word_given_prod)


print(prob_prod_group.shape)
print(prob_word_prod.shape)

(5, 3659)
(3659, 1992)


#### 2.2.7 Obtaining the Probability of Word Given Group

In [54]:
prob_word_group_mat = np.dot(prob_prod_group, prob_word_prod)

prob_word_group_df = pd.DataFrame(prob_word_group_mat, columns = x4.columns, index = ["Group 1", "Group 2", "Group 3", "Group 4", "Group 5"])

prob_word_group_df

,abc,abstract,acapulco,account,acrylic,address,ade,...,yellowblue,yelloworange,yellowpink,youre,yuletide,zinc,zincglass
Group 1,0.000072,0.000305,0.000091,1.495845e-04,0.001739,2.627733e-05,9.972514e-07,...,0.000029,0.000043,9.011270e-07,8.680650e-08,0.000003,0.002244,4.347798e-06
Group 2,0.000256,0.000116,0.001039,4.389257e-07,0.001106,2.169538e-07,2.453962e-06,...,0.000013,0.000054,1.328181e-04,2.279897e-04,0.000247,0.002403,7.240639e-07
Group 3,0.000342,0.000049,0.001089,1.817843e-04,0.000296,4.760089e-07,2.171213e-07,...,0.000028,0.000043,1.737323e-04,2.233463e-05,0.000046,0.002396,3.766509e-04
Group 4,0.000104,0.000282,0.000076,2.723137e-05,0.001810,1.527798e-04,6.495897e-07,...,0.000290,0.000150,3.461379e-07,1.483567e-05,0.000154,0.003775,6.313978e-07
Group 5,0.000002,0.000402,0.000024,9.273975e-05,0.001228,8.902907e-05,2.000178e-04,...,0.000011,0.000019,1.090101e-05,2.356254e-05,0.000058,0.002818,1.809188e-04


### 2.3 Finding the Top 5 Most Representative Word for Each Group

In [55]:
num_row, num_col = prob_word_group_df.shape

prob_word_group_final = []
prob_word_group_name = []
for j in range(num_row):
    prob_word_group_result = prob_word_group_df.iloc[j,:].sort_values(ascending = False)[:5]
    for results in prob_word_group_result:
        prob_word_group_final.append(results)
    for results in prob_word_group_result.index:
        prob_word_group_name.append(results)

table_val = np.transpose(np.vstack((prob_word_group_name, prob_word_group_final)))

index_name = ["1st Most Representative Word", 
            "2nd Most Representative Word",
            "3rd Most Representative Word",
            "4th Most Representative Word",
            "5th Most Representative Word"]

column_name = ["Word", "Probability"]


group1_result = table_val[0:5]
group1_result = pd.DataFrame(group1_result, columns = column_name, index = index_name)

group2_result = table_val[5:10]
group2_result = pd.DataFrame(group2_result, columns = column_name, index = index_name)

group3_result = table_val[10:15]
group3_result = pd.DataFrame(group3_result, columns = column_name, index = index_name)

group4_result = table_val[15:20]
group4_result = pd.DataFrame(group4_result, columns = column_name, index = index_name)

group5_result = table_val[20:25]
group5_result = pd.DataFrame(group5_result, columns = column_name, index = index_name)

#### 2.3.1 Top 5 Most Representative Words for Group 1

In [56]:
group1_result

,Word,Probability
1st Most Representative Word,set,0.024697779484951124
2nd Most Representative Word,pink,0.01712445972867032
3rd Most Representative Word,blue,0.014730535495957221
4th Most Representative Word,heart,0.013302303130797074
5th Most Representative Word,red,0.012786559657789623


#### 2.3.2 Top 5 Most Representative Words for Group 2

In [57]:
group2_result

,Word,Probability
1st Most Representative Word,set,0.020260544104559883
2nd Most Representative Word,pink,0.015305454785068968
3rd Most Representative Word,vintage,0.01457156857481174
4th Most Representative Word,heart,0.014395203749942402
5th Most Representative Word,red,0.013252228051531902


#### 2.3.3 Top 5 Most Representative Words for Group 3

In [58]:
group3_result

,Word,Probability
1st Most Representative Word,set,0.020233086994091886
2nd Most Representative Word,pink,0.017169284848418183
3rd Most Representative Word,bag,0.013903215579505385
4th Most Representative Word,heart,0.012877468986166637
5th Most Representative Word,vintage,0.011971310640774304


#### 2.3.4 Top 5 Most Representative Words for Group 4

In [59]:
group4_result

,Word,Probability
1st Most Representative Word,pink,0.019109123221444375
2nd Most Representative Word,set,0.017534726459877543
3rd Most Representative Word,blue,0.014610520616398387
4th Most Representative Word,heart,0.014571719704357788
5th Most Representative Word,vintage,0.012096163390454275


#### 2.3.5 Top 5 Most Representative Words for Group 5

In [60]:
group5_result

,Word,Probability
1st Most Representative Word,set,0.022905976356609153
2nd Most Representative Word,pink,0.019994107406958696
3rd Most Representative Word,blue,0.01406579092808226
4th Most Representative Word,vintage,0.013353235655880855
5th Most Representative Word,red,0.013153931865047113
